### 0. Setup Paths

In [1]:
import os

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('../data/Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('../data/Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('../data/Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('../data/Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('../data/Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('../data/Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('../data/Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('../data/Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('../data/Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('../data/Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('../data/Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('../data/Tensorflow','protoc')
 }

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('../data/Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [6]:
for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path) 

###  1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [12]:
# if os.name=='nt':
#     !pip install wget
import wget

In [8]:
# if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
#     !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into '../data/Tensorflow\models'...
Updating files:   4% (174/3872)
Updating files:   5% (194/3872)
Updating files:   6% (233/3872)
Updating files:   7% (272/3872)
Updating files:   8% (310/3872)
Updating files:   8% (314/3872)
Updating files:   9% (349/3872)
Updating files:  10% (388/3872)
Updating files:  11% (426/3872)
Updating files:  11% (442/3872)
Updating files:  12% (465/3872)
Updating files:  13% (504/3872)
Updating files:  14% (543/3872)
Updating files:  15% (581/3872)
Updating files:  15% (611/3872)
Updating files:  16% (620/3872)
Updating files:  17% (659/3872)
Updating files:  18% (697/3872)
Updating files:  19% (736/3872)
Updating files:  20% (775/3872)
Updating files:  20% (778/3872)
Updating files:  21% (814/3872)
Updating files:  22% (852/3872)
Updating files:  23% (891/3872)
Updating files:  24% (930/3872)
Updating files:  25% (968/3872)
Updating files:  25% (998/3872)
Updating files:  26% (1007/3872)
Updating files:  27% (1046/3872)
Updating files:  28% (1085

In [18]:
import os
import zipfile
import requests
import subprocess

# Define paths
PROTOC_PATH = paths['PROTOC_PATH']
RESEARCH_PATH = os.path.join('../data/Tensorflow', 'models', 'research')

if os.name == 'nt':
    # Download Protobuf
    url = "https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    response = requests.get(url)
    zip_path = os.path.join(PROTOC_PATH, 'protoc-3.15.6-win64.zip')

    # Save the downloaded zip file
    with open(zip_path, 'wb') as f:
        f.write(response.content)

    # Extract the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(PROTOC_PATH)

    # Update the PATH environment variable
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(PROTOC_PATH, 'bin'))

    # Compile the protobuf files
    subprocess.run(['protoc', os.path.join(RESEARCH_PATH, 'object_detection', 'protos', '*.proto'), '--python_out=.'], shell=True)

    # Copy setup.py
    setup_src = os.path.join(RESEARCH_PATH, 'object_detection', 'packages', 'tf2', 'setup.py')
    setup_dst = os.path.join(RESEARCH_PATH, 'setup.py')
    subprocess.run(['copy', setup_src, setup_dst], shell=True)

    # Install the package
    subprocess.run(['python', 'setup.py', 'build'], cwd=RESEARCH_PATH)
    subprocess.run(['python', 'setup.py', 'install'], cwd=RESEARCH_PATH)

    # Install slim
    slim_path = os.path.join(RESEARCH_PATH, 'slim')
    subprocess.run(['pip', 'install', '-e', '.'], cwd=slim_path)

In [16]:
# !pip install object_detection

  Using cached Cython-3.0.11-cp312-cp312-win_amd64.whl.metadata (3.2 kB)
  Using cached nbconvert-7.16.4-py3-none-any.whl.metadata (8.5 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached bleach-6.1.0-py3-none-any.whl.metadata (30 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached jupyterlab_pygments-0.3.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached mistune-3.0.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached nbclient-0.10.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached pandocfilters-1.5.1-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached tinycss2-1.3.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached jsonpointer-3.0.0-py2

In [21]:
# import os
# import zipfile
# import requests
# import subprocess

# # Define paths
# PROTOC_PATH = paths['PROTOC_PATH']
# RESEARCH_PATH = os.path.join('../data/Tensorflow', 'models', 'research')

# if os.name == 'nt':
#     # Download Protobuf
#     url = "https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
#     response = requests.get(url)
#     zip_path = os.path.join(PROTOC_PATH, 'protoc-3.15.6-win64.zip')

#     # Save the downloaded zip file
#     with open(zip_path, 'wb') as f:
#         f.write(response.content)

#     # Extract the zip file
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#         zip_ref.extractall(PROTOC_PATH)

#     # Update the PATH environment variable
#     os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(PROTOC_PATH, 'bin'))

#     # Compile the protobuf files
#     subprocess.run(['protoc', os.path.join(RESEARCH_PATH, 'object_detection', 'protos', '*.proto'), '--python_out=.'], shell=True)

#     # Copy setup.py
#     setup_src = os.path.join(RESEARCH_PATH, 'object_detection', 'packages', 'tf2', 'setup.py')
#     setup_dst = os.path.join(RESEARCH_PATH, 'setup.py')
#     subprocess.run(['copy', setup_src, setup_dst], shell=True)

#     # Install the package
#     subprocess.run(['python', 'setup.py', 'build'], cwd=RESEARCH_PATH)
#     subprocess.run(['python', 'setup.py', 'install'], cwd=RESEARCH_PATH)

#     # Install slim
#     slim_path = os.path.join(RESEARCH_PATH, 'slim')
#     subprocess.run(['pip', 'install', '-e', '.'], cwd=slim_path)


In [22]:
# # Verification Script
# VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')

# # Verify Installation
# subprocess.run(['python', VERIFICATION_SCRIPT])

CompletedProcess(args=['python', '../data/Tensorflow\\models\\research\\object_detection\\builders\\model_builder_tf2_test.py'], returncode=1)

In [ ]:
import object_detection

In [7]:
import slim
slim.__version__

ModuleNotFoundError: No module named 'slim'